# Setup

## Imports

In [ ]:
import contextlib
import io
import numpy as np
import os
import pandas as pd

In [ ]:
import cartopy.crs as ccrs

In [ ]:
import yt
import unyt
import trident

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import palettable

In [ ]:
import verdict
import trove
import helpers

## Parameters

In [ ]:
ions = [
    'H I',
    'O I',
    'C II',
    'C III',
    'C IV',
    'N II',
    'N III',
    'Si II',
    'Si III',
    'Si IV',
    'N V',
    'O VI',
    'Mg II'
]
snr = 30

In [ ]:
pm = trove.link_params_to_config(
    helpers.CONFIG,
    script_id = 'nb.1',
    variation = 'm12i_md',
)

In [ ]:
qual_colors = palettable.cartocolors.qualitative.Vivid_10.mpl_colors

In [ ]:
sim = pm['variation']

## Load Halo Data

In [ ]:
halo_catalog_fn = 'halo_{}.hdf5'.format( pm['snum'] )
halo_catalog_fp = os.path.join( pm['rockstar_data_dir'], halo_catalog_fn )

In [ ]:
halo_data = verdict.Dict.from_hdf5( halo_catalog_fp )
index = halo_data['mass'].argmax()
center_ckpc = halo_data['position'][index]
center = center_ckpc / ( 1. + halo_data['snapshot:redshift'] )

## Load Simulation Data

In [ ]:
yt_sim_fp = os.path.join( pm['sim_data_dir'], 'snapdir_{:03d}'.format( pm['snum'] ) )
ds = yt.load( yt_sim_fp )

In [ ]:
kpc = ds.quan( 1, 'kpc' )

## Load Processed Data

In [ ]:
data_fp = os.path.join( pm['processed_data_dir'], 'data.h5' )
data = verdict.Dict.from_hdf5( data_fp, create_nonexistent=True )

# Sightline Coordinates

## One-time Calculations

In [ ]:
new_data = False

In [ ]:
data_key = 'total_angular_momentum'
try:
    jtot = data[sim][data_key]
except KeyError:
    
    new_data = True
        
    sp = ds.sphere( center * kpc , (10, "kpc"))
    jtot = sp.quantities.angular_momentum_vector( particle_type='PartType4' ).to( 'kpc * km / s' ).value

    data.setitem(sim, jtot, data_key, )

In [ ]:
data_key = 'galaxy_velocity'
try:
    galaxy_velocity = data[sim][data_key]
except KeyError:
    
    new_data = True
        
    sp_center = ds.sphere( center * kpc, ( 10, 'kpc' ) )
    galaxy_velocity = sp_center.quantities.bulk_velocity().to( 'km/s' ).value

    data.setitem(sim, galaxy_velocity, data_key, )

In [ ]:
if new_data:
    data.to_hdf5( data_fp )

## End Position

### Position

In [ ]:
zhat = jtot / np.linalg.norm( jtot )
xhat = np.cross( [ 1, 0, 0 ], zhat )
xhat /= np.linalg.norm( xhat )
yhat = np.cross( zhat, xhat )

In [ ]:
end = center + pm['sun_galactocentric_radius'] * xhat

### Velocity

In [ ]:
data_key = 'sun_relative_velocity'
try:
    sun_relative_velocity = data[sim][data_key]
except KeyError:
    
    new_data = True
    
    sp_sun = ds.sphere( end * kpc, ( 2, 'kpc' ) )
    sun_velocity = sp_sun.quantities.bulk_velocity( particle_type='PartType4' ).to( 'km/s' )
    sun_relative_velocity = ( sun_velocity - galaxy_velocity * unyt.km / unyt.s ).value
    
    data.setitem(sim, sun_relative_velocity, data_key, )

In [ ]:
if new_data:
    data.to_hdf5( data_fp )

## Start Positions

### Load On-Sky Coords

In [ ]:
coords_fp = os.path.join( pm['processed_data_dir'], 'skycoords.txt' )
skycoords = pd.read_csv( coords_fp, sep=', ' )

In [ ]:
# Add a test coordinate
testcoord = pd.Series( data={ 'QSO ID': 'test', 'latitude': 30., 'longitude': 0. } )
skycoords = skycoords.append( testcoord, ignore_index=True, )

In [ ]:
# Add a second test coordinate
testcoord = pd.Series( data={ 'QSO ID': 'test', 'latitude': 0., 'longitude': 30. } )
skycoords = skycoords.append( testcoord, ignore_index=True, )

#### Plot

In [ ]:
proj = ccrs.Mollweide()
img_proj = ccrs.PlateCarree()

In [ ]:
# Check we have the proper number of points in each quadrant
normal_person_longitude = 360 - skycoords['longitude']
above_plane = skycoords['latitude'] > 0.
below_plane = skycoords['latitude'] < 0.
left_of_center = ( normal_person_longitude > 180. ) & ( normal_person_longitude < 360. )
right_of_center = normal_person_longitude < 180.
assert ( above_plane & left_of_center ).sum() == 4
assert ( above_plane & right_of_center ).sum() == 3
assert ( below_plane & left_of_center ).sum() == 6
assert ( below_plane & right_of_center ).sum() == 5

In [ ]:
fig = plt.figure()
ax = plt.axes( projection=proj )

ax.scatter(
    normal_person_longitude,
    skycoords['latitude'],
    transform = img_proj
)

ax.gridlines()
delta_extent = 1
ax.set_extent([180-delta_extent,180+delta_extent,-90,90], crs=img_proj)

### Convert to Simulation Coordinates

#### Calculate

In [ ]:
# Unit vectors for on-sky coordinate system.
# xskyhat points from the sun to the center of the galaxy.
# zskyhat points parallel to the axis of total angular momentum.
# yskyhat (should) point to the left on a sky map.
# Dot these with the coordinates to get coordinates in the sky frame.
xskyhat = -xhat
zskyhat = zhat
yskyhat = np.cross( zskyhat, xskyhat )

In [ ]:
# Convert from longitude and latitude to spherical coords, in the original (non-sky) frame.
phi = ( skycoords['longitude'] ) * np.pi / 180.
theta = ( 90 - skycoords['latitude'] ) * np.pi / 180.

In [ ]:
unitcoords_sky = np.array([ np.cos( phi ) * np.sin( theta ), np.sin( phi ) * np.sin( theta ), np.cos( theta ) ])

In [ ]:
unitcoords = (
    unitcoords_sky[0][:,np.newaxis] * xskyhat +
    unitcoords_sky[1][:,np.newaxis] * yskyhat +
    unitcoords_sky[2][:,np.newaxis] * zskyhat
)

In [ ]:
start = end + unitcoords * pm['pathlength']

In [ ]:
fig = plt.figure()
ax = plt.gca()

halfwidth = 10

i = 0
j = 1
x_i = [ 'x', 'y', 'z' ][i]
x_j = [ 'x', 'y', 'z' ][j]

for start_ in start:
    ax.plot(
        [ start_[i], end[i] ],
        [ start_[j], end[j] ],
        color = 'k',
        # s = 100,
    )

ax.scatter(
    end[i],
    end[j],
    color = 'k',
    s = 100,
    zorder = 100,
)

ax.scatter(
    center[i],
    center[j],
    color = 'k',
    s = 100,
    zorder = 100,
    marker = '*',
)

# Unit vectors
unit_vector_colors = palettable.cartocolors.qualitative.Safe_10.mpl_colors
skyvecs = [ xskyhat, yskyhat, zskyhat ]
for ii, xihat in enumerate( [ xhat, yhat, zhat ] ):
    ax.plot(
        [ center[i], center[i] + xihat[i]*halfwidth/2 ],
        [ center[j], center[j] + xihat[j]*halfwidth/2 ],
        color = unit_vector_colors[ii],
        linewidth = 3,
    )
    
    ax.plot(
        [ end[i], end[i] + skyvecs[ii][i]*halfwidth/2 ],
        [ end[j], end[j] + skyvecs[ii][j]*halfwidth/2 ],
        color = unit_vector_colors[ii],
        linewidth = 3,
    )


ax.set_xlim( center[i] - halfwidth, center[i] + halfwidth )
ax.set_ylim( center[j] - halfwidth, center[j] + halfwidth )


#### Check

In [ ]:
# Check orientation above/below plane
dot_z = np.einsum( 'ij,j->i', start[above_plane] - end, jtot )
np.testing.assert_allclose( np.sign( dot_z ), 1. )
dot_z = np.einsum( 'ij,j->i', start[below_plane] - end, jtot )
np.testing.assert_allclose( np.sign( dot_z ), -1. )

In [ ]:
# Check orientation left/right
dot_y = np.einsum( 'ij,j->i', start[left_of_center] - end, yhat )
np.testing.assert_allclose( np.sign( dot_z ), -1. )
dot_z = np.einsum( 'ij,j->i', start[right_of_center] - end, yhat )
np.testing.assert_allclose( np.sign( dot_z ), 1. )

In [ ]:
# Check the second test coordinate dotted with yhat should be - pm['pathlength'] * sin( 30 ).
# This checks if the coordinates on the left are on the side of the galaxy rotating towards the viewer.
np.testing.assert_allclose( np.dot( start[-1] - end, yhat ), -pm['pathlength'] * 0.5 )
# Once checked, get rid of the test coord
start = np.delete( start, -1, axis=0 )

In [ ]:
# Check the test coordinate dotted with the zhat should be pm['pathlength'] * cos( 60 )
np.testing.assert_allclose( np.dot( start[-1] - end, zhat ), pm['pathlength'] * 0.5 )
# Once checked, get rid of the test coord
start = np.delete( start, -1, axis=0 )

## Save

In [ ]:
data.setitem( sim, xskyhat, 'galactic_frame', 'xhat', )
data.setitem( sim, yskyhat, 'galactic_frame', 'yhat', )
data.setitem( sim, zskyhat, 'galactic_frame', 'zhat', )

In [ ]:
data.setitem( sim, start, 'start' )
data.setitem( sim, end, 'end' )

In [ ]:
data.to_hdf5( data_fp )

# Generate Rays and Spectra

In [ ]:
# Objects for use
ldb = trident.LineDatabase( 'lines.txt' )

## Generate Rays

In [ ]:
for i, start_i in enumerate( start ):
    
    print( 'Generating sightline {:03d}...'.format( i ) )
    
    ray_dir = os.path.join( pm['data_dir'], 'sightlines' )
    os.makedirs( ray_dir, exist_ok=True )
    ray_fp = os.path.join( ray_dir, 'ray_{:03d}.h5'.format( i ) )
    
    ray = trident.make_simple_ray(
        ds,
        start_position = ( start_i * ds.quan( 1, 'kpc' ) ),
        end_position = ( end * ds.quan( 1, 'kpc' ) ),
        data_filename = ray_fp,
        lines = ions,
    )

## Generate Spectra

In [ ]:
# Waaaay too much output otherwise
yt.utilities.logger.disable_stream_logging()

In [ ]:
sg = trident.SpectrumGenerator( line_database=ldb )

In [ ]:
observables_fp = os.path.join( pm['data_dir'], 'sightlines', 'observables.h5' )
observables_data = verdict.Dict.from_hdf5( observables_fp, create_nonexistent=True )

In [ ]:
for i, start_i in enumerate( start ):
    
    print( 'Generating spectra {:03d}...'.format( i ) )
    
    ray_dir = os.path.join( pm['data_dir'], 'sightlines' )
    ray_fp = os.path.join( ray_dir, 'ray_{:03d}.h5'.format( i ) )
    ray = yt.load( ray_fp )
    
    trident.add_ion_fields(ray, ions=ions )
    
    # Without the redirection the logging may still happen despite asking it not to (nicely).... Not sure why
    f = io.StringIO()
    with contextlib.redirect_stderr( f ):
        # Make the spectrum
        sg.make_spectrum( ray, lines=ions, store_observables=True, min_tau=1e-4 )
            
    sg.apply_lsf()
    sg.add_gaussian_noise( snr )
    
    # Save
    sg.save_spectrum(
        os.path.join( ray_dir, 'spectrum_{:03d}.h5'.format( i ) )
    )
    sg.plot_spectrum(
        os.path.join( ray_dir, 'spectrum_{:03d}.png'.format( i ) )
    )

    observables_data.setitem( '{:03d}'.format( i ), sg.line_observables_dict,  )

In [ ]:
observables_data.to_hdf5( observables_fp )